In [ ]:
%matplotlib inline

from copy import deepcopy
from os.path import join
from pyleecan.Functions.load import load
from pyleecan.definitions import DATA_DIR
import matplotlib.pyplot as plt
import numpy as np

import os
from datetime import datetime

# Chargement de la machine de référence
ref_path = join(DATA_DIR, "Machine", "Tesla_model_3.json")
M_ref = load(ref_path)

In [ ]:

print("=" * 70)
print("GÉNÉRATION DE VARIANTES TESLA MODEL 3 - MACHINE HAUTE PERFORMANCE")
print("=" * 70)

# 2) Calcul de variations adaptées à cette machine spécifique
print("\n CALCUL DE VARIATIONS ADAPTÉES")
print("=" * 40)

# Entrefer très petit (0.45 mm) - variations très limitées
airgap_tolerance = 0.05e-3  # 0.05 mm seulement
print(f" Entrefer (0.45 mm): Tolérance ±{airgap_tolerance*1000:.2f} mm")

# Aimants - 1 aimant par pôle, variations critiques
magnet_tolerance = 0.02  # 2% seulement
print(f" Aimants (6 aimants): Tolérance ±{magnet_tolerance*100:.0f}%")

# Bobinage - 15 tours, variations limitées
winding_tolerance = 1  # 1 tour seulement
print(f" Bobinage (15 tours): Tolérance ±{winding_tolerance} tour")

# Encoches - 54 encoches, variations modérées
slot_tolerance = 0.01  # 1% seulement
print(f" Encoches (54 encoches): Tolérance ±{slot_tolerance*100:.0f}%")

# Excentricité - très critique avec entrefer de 0.45 mm
eccentricity_max = 0.1e-3  # 0.1 mm maximum
print(f" Excentricité: Maximum {eccentricity_max*1000:.1f} mm")

# 3) Fonction de validation adaptée
def validate_tesla_machine(machine, name):
    """Validation spécifique pour machine Tesla haute performance"""
    errors = []
    warnings = []
    
    # Vérification entrefer critique
    airgap = machine.stator.Rint - machine.rotor.Rext
    if airgap <= 0:
        errors.append(f"Entrefer négatif: {airgap*1000:.2f} mm")
    elif airgap < 0.3e-3:
        errors.append(f"Entrefer trop petit: {airgap*1000:.2f} mm")
    elif airgap > 0.8e-3:
        errors.append(f"Entrefer trop grand: {airgap*1000:.2f} mm")
    
    # Vérification bobinage
    if hasattr(machine.stator.winding, 'Ntcoil'):
        if machine.stator.winding.Ntcoil < 10:
            warnings.append(f"Tours par bobine faibles: {machine.stator.winding.Ntcoil}")
        elif machine.stator.winding.Ntcoil > 20:
            warnings.append(f"Tours par bobine élevés: {machine.stator.winding.Ntcoil}")
    
    return errors, warnings

# 4) Création des variantes
print("\n CRÉATION DES VARIANTES")
print("=" * 30)

machines = []
names = []

# === VARIANTES SAINES (tolérances de fabrication) ===
print("\n VARIANTES SAINES:")

# S1: aimants +2% en épaisseur
M1 = deepcopy(M_ref)
for hole in M1.rotor.hole:
    for attr_name in dir(hole):
        if attr_name.startswith('magnet_'):
            magnet = getattr(hole, attr_name)
            if magnet is not None and hasattr(magnet, 'Hmag'):
                magnet.Hmag *= (1 + magnet_tolerance)
machines.append(M1)
names.append("Tesla_Model3_Saine_Aimants_Epaisseur_Plus2_Pourcent")
print(f"    S1: Épaisseur aimants +{magnet_tolerance*100:.0f}%")

# S2: aimants -2% en largeur
M2 = deepcopy(M_ref)
for hole in M2.rotor.hole:
    for attr_name in dir(hole):
        if attr_name.startswith('magnet_'):
            magnet = getattr(hole, attr_name)
            if magnet is not None and hasattr(magnet, 'Wmag'):
                magnet.Wmag *= (1 - magnet_tolerance)
machines.append(M2)
names.append("Tesla_Model3_Saine_Aimants_Largeur_Moins2_Pourcent")
print(f"    S2: Largeur aimants -{magnet_tolerance*100:.0f}%")

# S3: ouverture d'encoche stator +1%
M3 = deepcopy(M_ref)
if hasattr(M3.stator.slot, 'W0'):
    M3.stator.slot.W0 *= (1 + slot_tolerance)
machines.append(M3)
names.append("Tesla_Model3_Saine_Encoche_Stator_Plus1_Pourcent")
print(f"    S3: Ouverture encoche +{slot_tolerance*100:.0f}%")

# S4: entrefer +0.05mm (réduction rayon rotor)
M4 = deepcopy(M_ref)
M4.rotor.Rext *= (1 - airgap_tolerance/M_ref.rotor.Rext)
machines.append(M4)
names.append("Tesla_Model3_Saine_Entrefer_Plus0p05mm")
print(f"   S4: Entrefer +{airgap_tolerance*1000:.2f} mm")

# S5: plus de spires par dent (+1 tour)
M5 = deepcopy(M_ref)
if hasattr(M5.stator.winding, 'Ntcoil'):
    M5.stator.winding.Ntcoil += winding_tolerance
machines.append(M5)
names.append("Tesla_Model3_Saine_Spires_Plus1_Tour")
print(f"    S5: Tours +{winding_tolerance}")

# === VARIANTES DÉFAUTS (défauts de fabrication) ===
print("\n VARIANTES DÉFAUTS:")

# D1: excentricité statique - Simulation par décalage du centre du rotor
M6 = deepcopy(M_ref)
if hasattr(M6.rotor, 'hole') and M6.rotor.hole:
    for hole in M6.rotor.hole:
        if hasattr(hole, 'alpha'):
            hole.alpha += eccentricity_max / M6.rotor.Rext
machines.append(M6)
names.append("Tesla_Model3_Defaut_Excentricite_Statique_0p1mm")
print(f"    D1: Excentricité statique {eccentricity_max*1000:.1f}mm (simulée)")

# D2: excentricité dynamique (1° - limite critique)
M7 = deepcopy(M_ref)
if hasattr(M7.rotor, 'alpha'):
    M7.rotor.alpha = getattr(M7.rotor, "alpha", 0.0) + np.radians(1.0)
else:
    for hole in M7.rotor.hole:
        if hasattr(hole, 'alpha'):
            hole.alpha += np.radians(1.0)
machines.append(M7)
names.append("Tesla_Model3_Defaut_Excentricite_Dynamique_1deg")
print(f"   D2: Excentricité dynamique 1.0°")

# D3: démagnétisation partielle (5% - limite critique)
M8 = deepcopy(M_ref)
# Accès aux matériaux via les aimants dans les trous du rotor
for hole in M8.rotor.hole:
    for attr_name in dir(hole):
        if attr_name.startswith('magnet_'):
            magnet = getattr(hole, attr_name)
            if magnet is not None and hasattr(magnet, 'mat_type'):
                if hasattr(magnet.mat_type, 'mag') and hasattr(magnet.mat_type.mag, 'Br'):
                    original_br = magnet.mat_type.mag.Br
                    magnet.mat_type.mag.Br *= 0.95  # -5%
                    print(f"   D3: Br modifié de {original_br:.2f} T à {magnet.mat_type.mag.Br:.2f} T")
                    break
machines.append(M8)
names.append("Tesla_Model3_Defaut_Demagnetisation_5_Pourcent")
print(f"   D3: Démagnétisation 5%")

# D4: court-circuit spires local (1 tour - limite critique)
M9 = deepcopy(M_ref)
if hasattr(M9.stator.winding, 'Ntcoil'):
    original_turns = M9.stator.winding.Ntcoil
    M9.stator.winding.Ntcoil = max(1, M9.stator.winding.Ntcoil - 1)
    print(f"   D4: Tours modifiés de {original_turns} à {M9.stator.winding.Ntcoil}")
machines.append(M9)
names.append("Tesla_Model3_Defaut_Court_Circuit_Moins1_Tour")
print(f"   D4: Court-circuit -1 tour")

# D5: défaut usinage encoche (2% - limite critique)
M10 = deepcopy(M_ref)
if hasattr(M10.stator.slot, 'W0'):
    original_width = M10.stator.slot.W0
    M10.stator.slot.W0 *= 1.02  # +2%
    print(f"   D5: Largeur encoche modifiée de {original_width*1000:.2f} mm à {M10.stator.slot.W0*1000:.2f} mm")
machines.append(M10)
names.append("Tesla_Model3_Defaut_Usinage_Encoche_Plus2_Pourcent")
print(f"   D5: Défaut usinage +2%")

# 5) Validation des variantes
print(f"\n VALIDATION DES VARIANTES")
print("=" * 35)

valid_machines = []
valid_names = []

for Mi, nm in zip(machines, names):
    print(f"\n--- Validation de {nm} ---")
    
    # Validation spécifique Tesla
    errors, warnings = validate_tesla_machine(Mi, nm)
    
    if errors:
        print(f" ERREURS critiques:")
        for error in errors:
            print(f"   - {error}")
        print(f"   → Variante rejetée")
        continue
    
    if warnings:
        print(f"  Avertissements:")
        for warning in warnings:
            print(f"   - {warning}")
    
    print(f" {nm}: Validation réussie")
    valid_machines.append(Mi)
    valid_names.append(nm)

# 6) Création du dossier et sauvegarde des machines valides
print(f"\ CRÉATION DU DOSSIER ET SAUVEGARDE")
print("=" * 50)

# Création du dossier de sauvegarde dans le répertoire actuel
save_dir = "Tesla_Model3_Variantes"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    print(f" Dossier créé: {os.path.abspath(save_dir)}/")
else:
    print(f" Dossier existant: {os.path.abspath(save_dir)}/")

# Sous-dossiers organiser
saine_dir = os.path.join(save_dir, "Variantes_Saines")
defaut_dir = os.path.join(save_dir, "Variantes_Defauts")

os.makedirs(saine_dir, exist_ok=True)
os.makedirs(defaut_dir, exist_ok=True)

print(f" Sous-dossiers créés:")
print(f"   - {os.path.abspath(saine_dir)}/")
print(f"   - {os.path.abspath(defaut_dir)}/")

if valid_machines:
    print(f"\n SAUVEGARDE DES MACHINES VALIDES")
    print("=" * 40)
    
    # Sauvegarde de la machine de référence
    ref_path = os.path.join(save_dir, "Tesla_Model3_Reference.json")
    M_ref.save(ref_path)
    print(f" Référence sauvegardée: Tesla_Model3_Reference.json")
    
    # Compteurs pour les statistiques
    saine_count = 0
    defaut_count = 0
    
    for Mi, nm in zip(valid_machines, valid_names):
        # Déterminer le sous-dossier selon le type
        if "Saine" in nm:
            sub_dir = saine_dir
            saine_count += 1
        else:
            sub_dir = defaut_dir
            defaut_count += 1
        
        # Chemin de sauvegarde
        out_path = os.path.join(sub_dir, nm + ".json")
        
        # Sauvegarde
        Mi.save(out_path)
        print(f" Sauvegardé: {nm}")
        
        # Affichage de la machine
        fig, ax = Mi.plot(is_show_fig=False)
        plt.title(nm, fontsize=10)
        plt.show()
    
    # Rapport de sauvegarde
    print(f"\n RAPPORT DE SAUVEGARDE")
    print("=" * 30)
    print(f" Total machines sauvegardées: {len(valid_names)}")
    print(f"   - Variantes saines: {saine_count}")
    print(f"   - Variantes défauts: {defaut_count}")
    print(f"   - Machine de référence: 1")
    
    # Vérification des fichiers créés
    print(f"\n VÉRIFICATION DES FICHIERS")
    print("=" * 30)
    
    total_files = 0
    total_size = 0
    for root, dirs, files in os.walk(save_dir):
        for file in files:
            if file.endswith('.json'):
                file_path = os.path.join(root, file)
                file_size = os.path.getsize(file_path)
                total_files += 1
                total_size += file_size
                print(f" {file} - {file_size:,} octets")
    
    print(f"\n SUCCÈS: {total_files} fichiers JSON créés")
    print(f" Taille totale: {total_size:,} octets ({total_size/1024:.1f} KB)")
    
    # Affichage du chemin complet
    print(f"\n EMPLACEMENT FINAL:")
    print(f"   {os.path.abspath(save_dir)}/")
    
else:
    print(" Aucune machine valide générée")

print("\n" + "=" * 70)
print("GÉNÉRATION TERMINÉE")
print("=" * 70)

--------------------- NOUVEAUX TYPES DE DÉFAUTS À CIBLER-----------------------------------

In [ ]:
print("=" * 80)
print("TESLA MODEL 3 - VARIATIONS (100 MACHINES)")
print("=" * 80)

# 2) Analyse de la machine de référence pour définir les tolérances
print("\n ANALYSE POUR DÉFINIR LES TOLÉRANCES COHÉRENTES")
print("=" * 55)

# Valeurs de référence
airgap_ref = M_ref.stator.Rint - M_ref.rotor.Rext  # 0.45 mm
turns_ref = M_ref.stator.winding.Ntcoil  # 15 tours
rotor_radius = M_ref.rotor.Rext  # 74.85 mm

print(f" Entrefer de référence: {airgap_ref*1000:.2f} mm")
print(f" Tours de référence: {turns_ref}")
print(f" Rayon rotor: {rotor_radius*1000:.2f} mm")

# 3) Définition des tolérances cohérentes
print("\n TOLÉRANCES COHÉRENTES DÉFINIES")
print("=" * 40)

# Entrefer: ±10% maximum (0.045 mm)
airgap_tolerance_max = 0.1  # 10%
airgap_variations = np.linspace(-0.05, 0.05, 10)  # -5% à +5%

# Tours: ±20% maximum (3 tours)
turns_tolerance_max = 0.2  # 20%
turns_variations = [-3, -2, -1, 1, 2, 3]  # -3 à +3 tours

# Excentricité: ±20% de l'entrefer maximum (0.09 mm)
eccentricity_max = airgap_ref * 0.2  # 0.09 mm
eccentricity_variations = np.linspace(0.01e-3, 0.08e-3, 15)  # 0.01 à 0.08 mm

# Démagnétisation: ±15% maximum
demag_max = 0.15  # 15%
demag_variations = np.linspace(0.02, 0.15, 15)  # 2% à 15%

# Court-circuit: ±20% maximum (3 tours)
short_circuit_max = int(turns_ref * 0.2)  # 3 tours
short_circuit_variations = list(range(1, 4))  # 1 à 3 tours

# Défauts usinage: ±10% maximum
machining_max = 0.1  # 10%
machining_variations = np.linspace(-0.1, 0.1, 21)  # -10% à +10% (21 variations)

print(f" Entrefer: ±{airgap_tolerance_max*100:.0f}% (max ±{airgap_ref*airgap_tolerance_max*1000:.1f}mm)")
print(f" Tours: ±{turns_tolerance_max*100:.0f}% (max ±{int(turns_ref*turns_tolerance_max)} tours)")
print(f" Excentricité: max {eccentricity_max*1000:.1f}mm")
print(f" Démagnétisation: max {demag_max*100:.0f}%")
print(f" Court-circuit: max {short_circuit_max} tours")
print(f" Défauts usinage: ±{machining_max*100:.0f}%")

# 4) Fonction de validation adaptée
def validate_tesla_machine_coherent(machine, name):
    """Validation cohérente pour machine Tesla haute performance"""
    errors = []
    warnings = []
    
    # Vérification entrefer critique
    airgap = machine.stator.Rint - machine.rotor.Rext
    airgap_variation = abs(airgap - airgap_ref) / airgap_ref
    
    # VALIDATION SPÉCIALISÉE POUR EXCENTRICITÉ
    if "Excentricite" in name:
        # Pour les excentricités, accepter jusqu'à 60% de variation
        if airgap <= 0:
            errors.append(f"Entrefer négatif: {airgap*1000:.2f} mm")
        elif airgap < 0.1e-3:  # Minimum 0.1mm pour excentricité
            errors.append(f"Entrefer trop petit: {airgap*1000:.2f} mm")
        elif airgap_variation > 0.60:  # Maximum 60% pour excentricité
            errors.append(f"Variation entrefer excessive: {airgap_variation*100:.1f}%")
        elif airgap < 0.2e-3:  # Alerte si très petit
            warnings.append(f"Entrefer très petit: {airgap*1000:.2f} mm")
    
    # VALIDATION NORMALE POUR AUTRES DÉFAUTS
    else:
        if airgap <= 0:
            errors.append(f"Entrefer négatif: {airgap*1000:.2f} mm")
        elif airgap_variation > airgap_tolerance_max:  # 15% pour autres défauts
            errors.append(f"Variation entrefer excessive: {airgap_variation*100:.1f}%")
        elif airgap < 0.3e-3:
            warnings.append(f"Entrefer très petit: {airgap*1000:.2f} mm")
    
    # Vérification bobinage
    if hasattr(machine.stator.winding, 'Ntcoil'):
        turns_variation = abs(machine.stator.winding.Ntcoil - turns_ref) / turns_ref
        if turns_variation > turns_tolerance_max:
            errors.append(f"Variation tours excessive: {turns_variation*100:.1f}%")
        elif machine.stator.winding.Ntcoil < 5:
            errors.append(f"Tours par bobine trop faibles: {machine.stator.winding.Ntcoil}")
    
    return errors, warnings

# 5) Création des variantes cohérentes
print("\n CRÉATION DES VARIANTES COHÉRENTES")
print("=" * 45)

machines = []
names = []

# === VARIANTES SAINES COHÉRENTES (30 variantes) ===
print("\n VARIANTES SAINES COHÉRENTES (30 variantes):")

# Groupe 1: Variations d'aimants cohérentes (10 variantes) 
print("\n   Groupe 1: Variations d'aimants (±1% à ±5%)")
magnet_variations = np.linspace(-0.05, 0.05, 10)  # -5% à +5%

for i, var in enumerate(magnet_variations):
    M = deepcopy(M_ref)
    
    print(f"   Machine {i+1}: Application variation aimants {var*100:+.1f}%")
    
    # MODIFICATION DES DIMENSIONS GÉOMÉTRIQUES
    for hole in M.rotor.hole:
        for attr_name in dir(hole):
            if attr_name.startswith('magnet_'):
                magnet = getattr(hole, attr_name)
                if magnet is not None:
                    # Dimensions géométriques
                    if hasattr(magnet, 'Hmag'):
                        original_Hmag = magnet.Hmag
                        magnet.Hmag *= (1 + var)
                        print(f"  {attr_name} Hmag: {original_Hmag*1000:.2f}mm -> {magnet.Hmag*1000:.2f}mm")
                    
                    if hasattr(magnet, 'Wmag'):
                        original_Wmag = magnet.Wmag
                        magnet.Wmag *= (1 + var)
                        print(f"  {attr_name} Wmag: {original_Wmag*1000:.2f}mm -> {magnet.Wmag*1000:.2f}mm")
    
    # MODIFICATION DES PROPRIÉTÉS MAGNÉTIQUES ------
    for hole in M.rotor.hole:
        if hasattr(hole, 'magnet_dict') and hole.magnet_dict:
            for magnet_name, magnet in hole.magnet_dict.items():
                if magnet is not None and hasattr(magnet, 'mat_type'):
                    if hasattr(magnet.mat_type, 'mag'):
                        mag = magnet.mat_type.mag
                        
                        # Modification de la rémanence Br (IMPACT MAJEUR sur le couple)
                        if hasattr(mag, 'Brm20'):
                            original_Br = mag.Brm20
                            mag.Brm20 *= (1 + var * 0.8)  # 80% de la variation géométrique
                            print(f"  {magnet_name} Brm20: {original_Br:.4f}T -> {mag.Brm20:.4f}T")
                        
                        # Modification de Br également 
                        if hasattr(mag, 'Br'):
                            original_Br = mag.Br
                            mag.Br *= (1 + var * 0.8)
                            print(f"  {magnet_name} Br: {original_Br:.4f}T -> {mag.Br:.4f}T")
                        
                        # Modification du champ coercitif Hc (proportionnelle)
                        if hasattr(mag, 'Hc'):
                            original_Hc = mag.Hc
                            mag.Hc *= (1 + var * 0.6)  # 60% de la variation géométrique
                            print(f" {magnet_name} Hc: {original_Hc:.0f}A/m -> {mag.Hc:.0f}A/m")
                        
                        # Modification de la perméabilité relative
                        if hasattr(mag, 'mur_lin'):
                            original_mur = mag.mur_lin
                            mag.mur_lin *= (1 + var * 0.3)  # 30% de la variation géométrique
                            print(f"  {magnet_name} μr: {original_mur:.2f} -> {mag.mur_lin:.2f}")
    
    # VALIDATION GÉOMÉTRIQUE
    try:
        if hasattr(M.stator, 'slot'):
            M.stator.slot.build_geometry()
        print(f"   Géométrie valide")
    except Exception as e:
        print(f"   Géométrie invalide: {e}")
        continue
    
    machines.append(M)
    
    # Noms basés sur la variation
    sign = "Plus" if var > 0 else "Moins"
    name = f"Tesla_Model3_Saine_Aimants_{sign}{abs(var)*100:.1f}_Pourcent"
    names.append(name)
    print(f"    S{i+1:2d}: Aimants {sign}{abs(var)*100:.1f}% - {name}")
    print()

# Groupe 2: Variations d'encoches cohérentes (10 variantes) - STRATÉGIE ADAPTATIVE
print("\n   Groupe 2: Variations d'encoches (±0.5% à ±3%)")
slot_variations = np.linspace(-0.03, 0.03, 10)  # -3% à +3%

for i, var in enumerate(slot_variations):
    M = deepcopy(M_ref)
    
    print(f"   Machine {i+1}: Application variation encoches {var*100:+.1f}%")
    
    # CALCUL DU COEFFICIENT ADAPTATIF
    # Plus la variation est petite, plus le coefficient est grand
    if abs(var) < 0.001:  # Éviter division par zéro
        amplification = 50.0  # Coefficient maximum
    else:
        amplification = min(50.0, 0.03 / abs(var))  # Inverse proportionnel
    
    print(f"     Coefficient d'amplification: {amplification:.1f}x")
    
    # 1. MODIFICATION DU BOBINAGE (ADAPTATIF)
    if hasattr(M.stator.winding, 'Ntcoil'):
        original_turns = M.stator.winding.Ntcoil
        # Amplifier selon la variation
        turns_variation = int(original_turns * var * amplification * 0.1)
        M.stator.winding.Ntcoil = max(1, original_turns + turns_variation)
        
        if turns_variation != 0:
            print(f"     Tours bobinage: {original_turns} -> {M.stator.winding.Ntcoil} ({turns_variation:+d})")
    
    # 2. MODIFICATION DE LA PERMÉABILITÉ (ADAPTATIF)
    if hasattr(M.stator, 'mat_type') and M.stator.mat_type is not None:
        if hasattr(M.stator.mat_type, 'mag'):
            stator_mag = M.stator.mat_type.mag
            
            if hasattr(stator_mag, 'mur_lin'):
                original_mur = stator_mag.mur_lin
                # Amplifier selon la variation
                stator_mag.mur_lin *= (1 - var * amplification * 0.05)
                print(f"     Stator μr: {original_mur:.2f} -> {stator_mag.mur_lin:.2f}")
            
            if hasattr(stator_mag, 'Hc'):
                original_Hc = stator_mag.Hc
                stator_mag.Hc *= (1 + var * amplification * 0.03)
                print(f"     Stator Hc: {original_Hc:.0f}A/m -> {stator_mag.Hc:.0f}A/m")
    
    # 3. MODIFICATION DE L'ENTREFER (ADAPTATIF)
    if hasattr(M.stator, 'Rint') and hasattr(M.rotor, 'Rext'):
        original_airgap = M.stator.Rint - M.rotor.Rext
        # Amplifier selon la variation
        airgap_variation = original_airgap * var * amplification * 0.01
        M.stator.Rint += airgap_variation
        new_airgap = M.stator.Rint - M.rotor.Rext
        print(f"     Entrefer: {original_airgap*1000:.3f}mm -> {new_airgap*1000:.3f}mm")
    
    # 4. MODIFICATION DES PROPRIÉTÉS MAGNÉTIQUES DES AIMANTS (NOUVEAU !)
    # Pour les petites variations, modifier aussi les aimants
    if abs(var) < 0.01:  # Si variation < 1%
        print(f"     Petite variation détectée: modification des aimants")
        
        for hole in M.rotor.hole:
            if hasattr(hole, 'magnet_dict') and hole.magnet_dict:
                for magnet_name, magnet in hole.magnet_dict.items():
                    if magnet is not None and hasattr(magnet, 'mat_type'):
                        if hasattr(magnet.mat_type, 'mag'):
                            mag = magnet.mat_type.mag
                            
                            # Modification proportionnelle à l'amplification
                            if hasattr(mag, 'Brm20'):
                                original_Br = mag.Brm20
                                mag.Brm20 *= (1 + var * amplification * 0.05)
                                print(f"     {magnet_name} Brm20: {original_Br:.4f}T -> {mag.Brm20:.4f}T")
                            
                            if hasattr(mag, 'Hc'):
                                original_Hc = mag.Hc
                                mag.Hc *= (1 + var * amplification * 0.03)
                                print(f"     {magnet_name} Hc: {original_Hc:.0f}A/m -> {mag.Hc:.0f}A/m")
    
    # VALIDATION GÉOMÉTRIQUE
    try:
        if hasattr(M.stator, 'slot'):
            M.stator.slot.build_geometry()
        print(f"  Géométrie valide")
    except Exception as e:
        print(f"  Géométrie invalide: {e}")
        continue
    
    machines.append(M)
    
    sign = "Plus" if var > 0 else "Moins"
    name = f"Tesla_Model3_Saine_Encoches_{sign}{abs(var)*100:.1f}_Pourcent"
    names.append(name)
    print(f"   S{i+11:2d}: Encoches {sign}{abs(var)*100:.1f}% - {name}")
    print()
# Groupe 3: Variations d'entrefer cohérentes (5 variantes)
print("\n   Groupe 3: Variations d'entrefer (±2% à ±5%)")
airgap_variations = np.linspace(-0.05, 0.05, 5)  # -5% à +5%

for i, var in enumerate(airgap_variations):
    M = deepcopy(M_ref)
    M.rotor.Rext *= (1 - var)
    
    sign = "Plus" if var > 0 else "Moins"
    machines.append(M)
    names.append(f"Tesla_Model3_Saine_Entrefer_{sign}{abs(var)*100:.1f}_Pourcent")
    print(f"   S{i+21:2d}: Entrefer {sign}{abs(var)*100:.1f}%")

# Groupe 4: Variations de bobinage cohérentes (5 variantes)
print("\n   Groupe 4: Variations de bobinage (±1 à ±3 tours)")
winding_variations = [-3, -1, 1, 2, 3]

for i, var in enumerate(winding_variations):
    M = deepcopy(M_ref)
    if hasattr(M.stator.winding, 'Ntcoil'):
        M.stator.winding.Ntcoil += var
    
    sign = "Plus" if var > 0 else "Moins"
    machines.append(M)
    names.append(f"Tesla_Model3_Saine_Spires_{sign}{abs(var)}_Tours")
    print(f"   S{i+26:2d}: Tours {sign}{abs(var)}")

# === VARIANTES DÉFAUTS COHÉRENTES (69 variantes) ===
print("\n VARIANTES DÉFAUTS COHÉRENTES (69 variantes):")

# Groupe 7: Excentricités TRÈS AGRESSIVES pour visibilité maximale
print("\n   Groupe 7: Excentricités AGRESSIVES (40% à 60% de l'entrefer)")

# Entrefer de référence
airgap_ref = M_ref.stator.Rint - M_ref.rotor.Rext
print(f"   Entrefer de référence: {airgap_ref*1000:.2f} mm")

# Variations d'excentricité TRÈS importantes (40% à 60% de l'entrefer)  Passage de 0.1mm-2.0mm à 40%-60% de l'entrefer
eccentricity_percentages = np.linspace(0.40, 0.60, 15)  # 40% à 60% de l'entrefer
eccentricity_variations = eccentricity_percentages * airgap_ref  # Conversion en mm

print(f"   Excentricités à générer:")
for i, (pct, ecc) in enumerate(zip(eccentricity_percentages, eccentricity_variations)):
    print(f"     {i+1}. {pct*100:.0f}% de l'entrefer = {ecc*1000:.2f}mm")

for i, (pct, ecc) in enumerate(zip(eccentricity_percentages, eccentricity_variations)):
    M = deepcopy(M_ref)
    
    print(f"\n   Machine {i+1}: Excentricité {pct*100:.0f}% ({ecc*1000:.2f}mm)")
    
    # === EXCENTRICITÉ RÉELLE ET AGRESSIVE ===
    
    # 1. MODIFICATION DE LA POSITION DU ROTOR (excentricité statique)
    if hasattr(M.rotor, 'x0'):
        M.rotor.x0 = ecc  # Déplacement en X
        print(f"     Rotor x0: 0.0mm -> {ecc*1000:.2f}mm")
    
    if hasattr(M.rotor, 'y0'):
        M.rotor.y0 = 0.0  # Pas de déplacement en Y
        print(f"     Rotor y0: 0.0mm -> 0.0mm")
    
    # 2. MODIFICATION DE L'ENTREFER (effet principal de l'excentricité)
    if hasattr(M.stator, 'Rint') and hasattr(M.rotor, 'Rext'):
        original_airgap = M.stator.Rint - M.rotor.Rext
        
        # L'excentricité modifie l'entrefer de façon non-uniforme
        # Entrefer minimum = entrefer_original - excentricité
        min_airgap = original_airgap - ecc
        max_airgap = original_airgap + ecc
        
        # Ajustement de rayon rotor pour simuler l'effet
        M.rotor.Rext = M.stator.Rint - min_airgap
        
        print(f"     Entrefer original: {original_airgap*1000:.2f}mm")
        print(f"     Entrefer minimum: {min_airgap*1000:.2f}mm")
        print(f"     Entrefer maximum: {max_airgap*1000:.2f}mm")
        print(f"     Variation entrefer: ±{ecc*1000:.2f}mm")
        
        # Vérification de cohérence
        if min_airgap <= 0:
            print(f"     ATTENTION: Entrefer minimum négatif !")
        elif min_airgap < 0.1e-3:  # Moins de 0.1mm
            print(f"     ATTENTION: Entrefer minimum très petit !")
    
    # 3. MODIFICATION DE LA POSITION DES AIMANTS (excentricité dynamique)
    if hasattr(M.rotor, 'hole') and M.rotor.hole:
        for hole_idx, hole in enumerate(M.rotor.hole):
            # déplace chaque trou d'aimant proportionnellement
            if hasattr(hole, 'x0'):
                hole.x0 = ecc * np.cos(hole.alpha)  # Position X du trou
                print(f"     Trou {hole_idx}: x0 = {hole.x0*1000:.2f}mm")
            
            if hasattr(hole, 'y0'):
                hole.y0 = ecc * np.sin(hole.alpha)  # Position Y du trou
                print(f"     Trou {hole_idx}: y0 = {hole.y0*1000:.2f}mm")
            
            # Modification de l'angle pour tenir compte de l'excentricité
            if hasattr(hole, 'alpha'):
                original_alpha = hole.alpha
                # L'excentricité modifie l'angle apparent
                hole.alpha += ecc * 1000  # Ajustement angulaire
                print(f"     Trou {hole_idx}: α = {original_alpha:.3f} -> {hole.alpha:.3f} rad")
    
    # 4. MODIFICATION DES PROPRIÉTÉS MAGNÉTIQUES (effet de l'excentricité)
    if hasattr(M.rotor, 'hole') and M.rotor.hole:
        for hole in M.rotor.hole:
            if hasattr(hole, 'magnet_dict') and hole.magnet_dict:
                for magnet_name, magnet in hole.magnet_dict.items():
                    if magnet is not None and hasattr(magnet, 'mat_type'):
                        if hasattr(magnet.mat_type, 'mag'):
                            mag = magnet.mat_type.mag
                            
                            # L'excentricité affecte les propriétés magnétiques
                            if hasattr(mag, 'Brm20'):
                                original_br = mag.Brm20
                                # Dégradation due à l'excentricité (plus importante)
                                mag.Brm20 *= (1 - ecc * 500)  # Dégradation proportionnelle
                                print(f"     {magnet_name} Brm20: {original_br:.4f}T -> {mag.Brm20:.4f}T")
                            
                            # Modification de la perméabilité relative
                            if hasattr(mag, 'mur_lin'):
                                original_mur = mag.mur_lin
                                mag.mur_lin *= (1 + ecc * 200)  # Augmentation de μr
                                print(f"     {magnet_name} μr: {original_mur:.2f} -> {mag.mur_lin:.2f}")
    
    # 5. MODIFICATION DU BOBINAGE (effet de l'excentricité sur le bobinage)
    if hasattr(M.stator.winding, 'Ntcoil'):
        original_turns = M.stator.winding.Ntcoil
        # L'excentricité peut affecter le bobinage
        M.stator.winding.Ntcoil = max(1, int(original_turns * (1 + ecc * 100)))  # ±0.1% tours
        print(f"     Tours bobinage: {original_turns} -> {M.stator.winding.Ntcoil}")
    
    # 6. MODIFICATION DE LA TENSION (VOLTAIRE) ------------------------------------------------
    # L'excentricité affecte la tension induite
    if hasattr(M.stator.winding, 'Ntcoil'):
        # La tension est proportionnelle au nombre de tours et au flux
        # L'excentricité modifie le flux, donc la tension
        voltage_factor = 1 + ecc * 300  # Facteur de tension
        print(f"     Facteur de tension: {voltage_factor:.3f}")
        
        # Modifie le nombre de tours pour simuler l'effet sur la tension
        M.stator.winding.Ntcoil = max(1, int(original_turns * voltage_factor))
        print(f"     Tours ajustés pour tension: {original_turns} -> {M.stator.winding.Ntcoil}")
    
    # VALIDATION GÉOMÉTRIQUE
    try:
        # Test de construction
        if hasattr(M.stator, 'slot'):
            M.stator.slot.build_geometry()
        print(f"     Géométrie valide")
    except Exception as e:
        print(f"      Géométrie invalide: {e}")
        continue
    
    machines.append(M)
    
    # Noms basés sur l'excentricité en pourcentage
    name = f"Tesla_Model3_Defaut_Excentricite_{pct*100:.0f}Pourcent_Entrefer"
    names.append(name)
    print(f"   D{i+30:2d}: Excentricité {pct*100:.0f}% ({ecc*1000:.2f}mm) - {name}")
    
    print()

# === RÉSUMÉ DES EXCENTRICITÉS GÉNÉRÉES ===
print(f"\n" + "=" * 80)
print("RÉSUMÉ DES EXCENTRICITÉS AGRESSIVES GÉNÉRÉES")
print("=" * 80)

print(f" Entrefer de référence: {airgap_ref*1000:.2f} mm")
print(f" Excentricités générées:")
for i, (pct, ecc) in enumerate(zip(eccentricity_percentages, eccentricity_variations)):
    min_airgap = airgap_ref - ecc
    max_airgap = airgap_ref + ecc
    print(f"   {i+1:2d}. {pct*100:2.0f}% = {ecc*1000:5.2f}mm -> Entrefer: {min_airgap*1000:.2f}mm à {max_airgap*1000:.2f}mm")

print(f"\n Modifications appliquées:")
print(f"   - Position du rotor (x0, y0)")
print(f"   - Rayon rotor (Rext)")
print(f"   - Position des aimants (x0, y0, α)")
print(f"   - Propriétés magnétiques (Br, μr)")
print(f"   - Bobinage (Ntcoil)")
print(f"   - Facteur de tension (voltaire)")

print(f"\n Total machines générées: {len(machines)}")
print("=" * 80)

# Groupe 6: Démagnétisations cohérentes (35 variantes) - ÉTENDU JUSQU'À 70%
print("\n   Groupe 6: Démagnétisations (2% à 70%)")
demag_variations = np.linspace(0.02, 0.70, 35)  # 2% à 70% (35 variations)

for i, demag in enumerate(demag_variations):
    M = deepcopy(M_ref)
    
    print(f"   Machine {i+1}: Application démagnétisation {demag*100:.1f}%")
    
    # Applique la démagnétisation sur les aimants
    if hasattr(M.rotor, 'hole') and M.rotor.hole:
        for hole in M.rotor.hole:
            # Accéde aux aimants via magnet_dict
            if hasattr(hole, 'magnet_dict') and hole.magnet_dict:
                for magnet_name, magnet in hole.magnet_dict.items():
                    if magnet is not None and hasattr(magnet, 'mat_type'):
                        if hasattr(magnet.mat_type, 'mag'):
                            mag = magnet.mat_type.mag
                            
                            # Modification de Brm20 (propriété principale) - TRÈS SENSIBLE
                            if hasattr(mag, 'Brm20'):
                                original_br = mag.Brm20
                                mag.Brm20 *= (1 - demag)
                                print(f"     {magnet_name}: Brm20 {original_br:.4f}T -> {mag.Brm20:.4f}T (-{demag*100:.1f}%)")
                            
                            # Modification de Br (si présent)
                            if hasattr(mag, 'Br'):
                                original_br = mag.Br
                                mag.Br *= (1 - demag)
                                print(f"     {magnet_name}: Br {original_br:.4f}T -> {mag.Br:.4f}T (-{demag*100:.1f}%)")
                            
                            # Modification de Hc (champ coercitif) - PROPORTIONNELLE
                            if hasattr(mag, 'Hc'):
                                original_hc = mag.Hc
                                mag.Hc *= (1 - demag * 0.8)  # Dégradation plus faible pour Hc
                                print(f"     {magnet_name}: Hc {original_hc:.0f}A/m -> {mag.Hc:.0f}A/m (-{demag*80:.1f}%)")
                            
                            # Modification de la perméabilité relative (effet de démagnétisation)
                            if hasattr(mag, 'mur_lin'):
                                original_mur = mag.mur_lin
                                mag.mur_lin *= (1 - demag * 0.3)  # Dégradation modérée de μr
                                print(f"     {magnet_name}: μr {original_mur:.2f} -> {mag.mur_lin:.2f} (-{demag*30:.1f}%)")
                            
                            # Modification de la densité (effet de porosité due à la démagnétisation)
                            if hasattr(mag, 'rho'):
                                original_rho = mag.rho
                                mag.rho *= (1 - demag * 0.1)  # Légère diminution de densité
                                print(f"     {magnet_name}: ρ {original_rho:.0f}kg/m³ -> {mag.rho:.0f}kg/m³ (-{demag*10:.1f}%)")
    
    # VALIDATION GÉOMÉTRIQUE
    try:
        # Test de construction
        if hasattr(M.stator, 'slot'):
            M.stator.slot.build_geometry()
        print(f"    Géométrie valide")
    except Exception as e:
        print(f"    Géométrie invalide: {e}")
        continue
    
    machines.append(M)
    
    # Noms basés sur le pourcentage de démagnétisation
    name = f"Tesla_Model3_Defaut_Demagnetisation_{demag*100:.0f}_Pourcent"
    names.append(name)
    print(f"   D{i+16:2d}: Démagnétisation {demag*100:.0f}% - {name}")
    
    print()

# Groupe 7: Court-circuits cohérentes (15 variantes) 
print("\n   Groupe 7: Court-circuits (1 à 3 tours)")
short_circuit_variations = list(range(1, 4)) * 5  # Répéter 5 fois

for i, short in enumerate(short_circuit_variations):
    M = deepcopy(M_ref)
    if hasattr(M.stator.winding, 'wind_mat'):
        M.stator.winding.wind_mat[0][0][0][0] = max(1, M.stator.winding.Ntcoil - short)
        # M.stator.winding.Ntcoil = max(1, M.stator.winding.Ntcoil - short)
    
    machines.append(M)
    names.append(f"Tesla_Model3_Defaut_Court_Circuit_{short}_Tours")
    print(f"    D{i+31:2d}: Court-circuit -{short} tours")

# Groupe 8: Défauts usinage cohérentes (21 variantes) 
print("\n   Groupe 8: Défauts usinage (-5% à +5%)")

# Limite des variations à ±5% pour éviter les problèmes FEMM
machining_variations_corrected = np.linspace(-0.05, 0.05, 21)  # -5% à +5% (21 variations)

for i, mach in enumerate(machining_variations_corrected):
    M = deepcopy(M_ref)
    
    print(f"   Machine {i+1}: Application défaut {mach*100:+.1f}%")
    
    #  paramètres géométriques et magnétiques proportionnelle
    
    # 1. MODIFICATION DE L'ENTREFER (très sensible aux défauts d'usinage)
    if hasattr(M.stator, 'Rint') and hasattr(M.rotor, 'Rext'):
        original_airgap = M.stator.Rint - M.rotor.Rext
        airgap_variation = original_airgap * mach * 0.2  # ±1% de l'entrefer
        M.rotor.Rext -= airgap_variation
        print(f"     Entrefer: {original_airgap*1000:.2f}mm -> {(original_airgap + airgap_variation)*1000:.2f}mm")
    
    # 2. MODIFICATION DES DIMENSIONS DES ENCOCHES (défaut d'usinage stator)
    if hasattr(M.stator, 'slot'):
        slot = M.stator.slot
        if hasattr(slot, 'W0'):
            original_W0 = slot.W0
            slot.W0 *= (1 + mach * 0.4)  # ±2% largeur d'ouverture
            print(f"     W0: {original_W0*1000:.2f}mm -> {slot.W0*1000:.2f}mm")
        
        if hasattr(slot, 'H0'):
            original_H0 = slot.H0
            slot.H0 *= (1 + mach * 0.3)  # ±1.5% profondeur d'ouverture
            print(f"     H0: {original_H0*1000:.2f}mm -> {slot.H0*1000:.2f}mm")
        
        if hasattr(slot, 'W1'):
            original_W1 = slot.W1
            slot.W1 *= (1 + mach * 0.3)  # ±1.5% largeur de base
            print(f"     W1: {original_W1*1000:.2f}mm -> {slot.W1*1000:.2f}mm")
        
        if hasattr(slot, 'H1'):
            original_H1 = slot.H1
            slot.H1 *= (1 + mach * 0.2)  # ±1% profondeur de base
            print(f"     H1: {original_H1*1000:.2f}mm -> {slot.H1*1000:.2f}mm")
    
    # 3. MODIFICATION DE LA LONGUEUR ACTIVE (défaut d'usinage axial)
    if hasattr(M.stator, 'L1'):
        original_L1 = M.stator.L1
        M.stator.L1 *= (1 + mach * 0.2)  # ±1% longueur active
        print(f"     L1: {original_L1*1000:.2f}mm -> {M.stator.L1*1000:.2f}mm")
    
    # 4. MODIFICATION DES DIMENSIONS DES AIMANTS (défaut d'usinage rotor)
    if hasattr(M.rotor, 'hole') and M.rotor.hole:
        for hole_idx, hole in enumerate(M.rotor.hole):
            if hasattr(hole, 'magnet_dict') and hole.magnet_dict:
                for magnet_name, magnet in hole.magnet_dict.items():
                    if magnet is not None:
                        if hasattr(magnet, 'Hmag'):
                            original_Hmag = magnet.Hmag
                            magnet.Hmag *= (1 + mach * 0.6)  # ±3% épaisseur aimant
                            print(f"     {magnet_name} Hmag: {original_Hmag*1000:.2f}mm -> {magnet.Hmag*1000:.2f}mm")
                        
                        if hasattr(magnet, 'Wmag'):
                            original_Wmag = magnet.Wmag
                            magnet.Wmag *= (1 + mach * 0.6)  # ±3% largeur aimant
                            print(f"     {magnet_name} Wmag: {original_Wmag*1000:.2f}mm -> {magnet.Wmag*1000:.2f}mm")
    
    # 5. MODIFICATION DES PROPRIÉTÉS MAGNÉTIQUES (effet de rugosité d'usinage)
    if hasattr(M.rotor, 'hole') and M.rotor.hole:
        for hole in M.rotor.hole:
            if hasattr(hole, 'magnet_dict') and hole.magnet_dict:
                for magnet_name, magnet in hole.magnet_dict.items():
                    if magnet is not None and hasattr(magnet, 'mat_type'):
                        if hasattr(magnet.mat_type, 'mag'):
                            mag = magnet.mat_type.mag
                            if hasattr(mag, 'Brm20'):
                                original_Brm20 = mag.Brm20
                                mag.Brm20 *= (1 - abs(mach) * 0.1)  # ±0.5% Br (dégradation)
                                print(f"     {magnet_name} Brm20: {original_Brm20:.2f}T -> {mag.Brm20:.2f}T")
                            
                            if hasattr(mag, 'mur_lin'):
                                original_mur_lin = mag.mur_lin
                                mag.mur_lin *= (1 + mach * 0.05)  # ±0.25% μr
                                print(f"     {magnet_name} μr: {original_mur_lin:.2f} -> {mag.mur_lin:.2f}")
    
    # 6. MODIFICATION DU BOBINAGE (défaut d'usinage bobinage)
    if hasattr(M.stator.winding, 'Ntcoil'):
        original_turns = M.stator.winding.Ntcoil
        M.stator.winding.Ntcoil = max(1, int(original_turns * (1 + mach * 0.2)))  # ±1% tours
        print(f"     Tours: {original_turns} -> {M.stator.winding.Ntcoil}")
    
    # VALIDATION GÉOMÉTRIQUE
    try:
        # Test de construction de la géométrie
        if hasattr(M.stator, 'slot'):
            M.stator.slot.build_geometry()
        print(f"     Géométrie valide")
    except Exception as e:
        print(f"     Géométrie invalide: {e}")
        continue
    
    machines.append(M)
    
    # Noms basés sur l'index afin d'eviter les doublant
    if mach < 0:
        name = f"Tesla_Model3_Defaut_Usinage_Moins{abs(mach)*100:.1f}_Pourcent"
    elif mach > 0:
        name = f"Tesla_Model3_Defaut_Usinage_Plus{mach*100:.1f}_Pourcent"
    else:
        name = "Tesla_Model3_Defaut_Usinage_Zero_Pourcent"
    
    names.append(name)
    print(f"  D{i+46:2d}: Défaut usinage {mach*100:+.1f}% - {name}")
    
    print()
    
# Groupe 9: Défauts matériau cohérentes (17 variantes) 
print("\n   Groupe 9: Défauts matériau (-8% à +8%)")

# Variations de défauts de matériau
material_variations = np.linspace(-0.08, 0.08, 17)  # -8% à +8% (17 variations)

for i, mat in enumerate(material_variations):
    M = deepcopy(M_ref)
    
    print(f"   Machine {i+1}: Application défaut matériau {mat*100:+.1f}%")
    
    # DÉFAUTS DE MATÉRIAU RÉALISTES : Modifie les propriétés magnétiques de façon significative
    
    # 1. MODIFICATION DES PROPRIÉTÉS MAGNÉTIQUES DES AIMANTS (TRÈS SENSIBLE)
    if hasattr(M.rotor, 'hole') and M.rotor.hole:
        for hole_idx, hole in enumerate(M.rotor.hole):
            if hasattr(hole, 'magnet_dict') and hole.magnet_dict:
                for magnet_name, magnet in hole.magnet_dict.items():
                    if magnet is not None and hasattr(magnet, 'mat_type'):
                        if hasattr(magnet.mat_type, 'mag'):
                            mag = magnet.mat_type.mag
                            
                            # Modification de la rémanence Br (IMPACT MAJEUR sur le couple)
                            if hasattr(mag, 'Brm20'):
                                original_Brm20 = mag.Brm20
                                mag.Brm20 *= (1 + mat * 0.6)  # ±4.8% Br (augmenté)
                                print(f"     {magnet_name} Brm20: {original_Brm20:.2f}T -> {mag.Brm20:.2f}T")
                            
                            # Modification de Br également (si présent)
                            if hasattr(mag, 'Br'):
                                original_Br = mag.Br
                                mag.Br *= (1 + mat * 0.6)  # ±4.8% Br
                                print(f"     {magnet_name} Br: {original_Br:.2f}T -> {mag.Br:.2f}T")
                            
                            # Modification du champ coercitif Hc
                            if hasattr(mag, 'Hc'):
                                original_Hc = mag.Hc
                                mag.Hc *= (1 + mat * 0.4)  # ±3.2% Hc (augmenté)
                                print(f"     {magnet_name} Hc: {original_Hc:.0f}A/m -> {mag.Hc:.0f}A/m")
                            
                            # Modification de la perméabilité relative
                            if hasattr(mag, 'mur_lin'):
                                original_mur_lin = mag.mur_lin
                                mag.mur_lin *= (1 + mat * 0.5)  # ±4% μr (augmenté)
                                print(f"     {magnet_name} μr: {original_mur_lin:.2f} -> {mag.mur_lin:.2f}")
                            
                            # Modification de la densité (effet de porosité)
                            if hasattr(mag, 'rho'):
                                original_rho = mag.rho
                                mag.rho *= (1 - abs(mat) * 0.3)  # ±2.4% densité (augmenté)
                                print(f"     {magnet_name} ρ: {original_rho:.0f}kg/m³ -> {mag.rho:.0f}kg/m³")
    
    # 2. MODIFICATION DES PROPRIÉTÉS DU MATÉRIAU FERROMAGNÉTIQUE (STATOR/ROTOR)
    if hasattr(M.stator, 'mat_type') and M.stator.mat_type is not None:
        if hasattr(M.stator.mat_type, 'mag'):
            stator_mag = M.stator.mat_type.mag
            if hasattr(stator_mag, 'mur_lin'):
                original_stator_mur = stator_mag.mur_lin
                stator_mag.mur_lin *= (1 + mat * 0.4)  # ±3.2% μr stator (augmenté)
                print(f"     Stator μr: {original_stator_mur:.2f} -> {stator_mag.mur_lin:.2f}")
            
            # Modification du champ coercitif du stator
            if hasattr(stator_mag, 'Hc'):
                original_stator_Hc = stator_mag.Hc
                stator_mag.Hc *= (1 + mat * 0.5)  # ±4% Hc stator (augmenté)
                print(f"     Stator Hc: {original_stator_Hc:.0f}A/m -> {stator_mag.Hc:.0f}A/m")
    
    if hasattr(M.rotor, 'mat_type') and M.rotor.mat_type is not None:
        if hasattr(M.rotor.mat_type, 'mag'):
            rotor_mag = M.rotor.mat_type.mag
            if hasattr(rotor_mag, 'mur_lin'):
                original_rotor_mur = rotor_mag.mur_lin
                rotor_mag.mur_lin *= (1 + mat * 0.4)  # ±3.2% μr rotor (augmenté)
                print(f"     Rotor μr: {original_rotor_mur:.2f} -> {rotor_mag.mur_lin:.2f}")
            
            # Modification du champ coercitif du rotor
            if hasattr(rotor_mag, 'Hc'):
                original_rotor_Hc = rotor_mag.Hc
                rotor_mag.Hc *= (1 + mat * 0.5)  # ±4% Hc rotor (augmenté)
                print(f"     Rotor Hc: {original_rotor_Hc:.0f}A/m -> {rotor_mag.Hc:.0f}A/m")
    
    # 3. MODIFICATION DE LA CONDUCTIVITÉ ÉLECTRIQUE (pertes par courants de Foucault)
    if hasattr(M.stator, 'mat_type') and M.stator.mat_type is not None:
        if hasattr(M.stator.mat_type, 'elec'):
            stator_elec = M.stator.mat_type.elec
            if hasattr(stator_elec, 'sigma'):
                original_sigma = stator_elec.sigma
                stator_elec.sigma *= (1 + mat * 0.3)  # ±2.4% conductivité (augmenté)
                print(f"     Stator σ: {original_sigma:.2e}S/m -> {stator_elec.sigma:.2e}S/m")
    
    # 4. MODIFICATION DES PERTES MAGNÉTIQUES (hystérésis et Foucault)
    if hasattr(M.stator, 'mat_type') and M.stator.mat_type is not None:
        if hasattr(M.stator.mat_type, 'mag'):
            stator_mag = M.stator.mat_type.mag
            # Pertes par hystérésis
            if hasattr(stator_mag, 'Hc'):
                original_stator_Hc = stator_mag.Hc
                stator_mag.Hc *= (1 + mat * 0.6)  # ±4.8% Hc stator (augmenté)
                print(f"     Stator Hc (hystérésis): {original_stator_Hc:.0f}A/m -> {stator_mag.Hc:.0f}A/m")
    
    # 5. MODIFICATION DES PROPRIÉTÉS DU BOBINAGE (effet de température)
    if hasattr(M.stator.winding, 'Ntcoil'):
        original_turns = M.stator.winding.Ntcoil
        # Légère modification due aux défauts de matériau du bobinage
        M.stator.winding.Ntcoil = max(1, int(original_turns * (1 + mat * 0.1)))  # ±0.8% tours
        print(f"     Tours bobinage: {original_turns} -> {M.stator.winding.Ntcoil}")
    
    # VALIDATION GÉOMÉTRIQUE
    try:
        # Test de construction
        if hasattr(M.stator, 'slot'):
            M.stator.slot.build_geometry()
        print(f"     Géométrie valide")
    except Exception as e:
        print(f"     Géométrie invalide: {e}")
        continue
    
    machines.append(M)
    
    # Noms basés sur l'index pour éviter les doublons
    if mat < 0:
        name = f"Tesla_Model3_Defaut_Materiau_Moins{abs(mat)*100:.1f}_Pourcent"
    elif mat > 0:
        name = f"Tesla_Model3_Defaut_Materiau_Plus{mat*100:.1f}_Pourcent"
    else:
        name = "Tesla_Model3_Defaut_Materiau_Zero_Pourcent"
    
    names.append(name)
    print(f"  D{i+63:2d}: Défaut matériau {mat*100:+.1f}% - {name}")
    
    print()

# 6) Validation et save
print(f"\n VALIDATION DES VARIANTES COHÉRENTES")
print("=" * 50)

valid_machines = []
valid_names = []

for Mi, nm in zip(machines, names):
    errors, warnings = validate_tesla_machine_coherent(Mi, nm)
    
    if errors:
        print(f" {nm}: REJETÉE - {errors[0]}")
        continue
    
    if warnings:
        print(f"  {nm}: {warnings[0]}")
    
    valid_machines.append(Mi)
    valid_names.append(nm)

print(f"\n RÉSULTATS DE VALIDATION COHÉRENTE:")
print(f"   Total créées: {len(machines)}")
print(f"   Validées: {len(valid_machines)}")
print(f"   Rejetées: {len(machines) - len(valid_machines)}")

# AFFICHAGE DE DÉTAILLÉ DES EXCENTRICITÉS VALIDÉES
print(f"\n DÉTAIL DES EXCENTRICITÉS VALIDÉES:")
eccentricity_validated = 0
for Mi, nm in zip(valid_machines, valid_names):
    if "Excentricite" in nm:
        eccentricity_validated += 1
        airgap = Mi.stator.Rint - Mi.rotor.Rext
        airgap_variation = abs(airgap - airgap_ref) / airgap_ref
        print(f"   {nm}: Entrefer {airgap*1000:.2f}mm (variation {airgap_variation*100:.1f}%)")

print(f"\n Excentricités validées: {eccentricity_validated}/15")

# 7) save data
print(f"\n SAUVEGARDE DU DATASET COHÉRENT")
print("=" * 40)

save_dir = "Tesla_Model3_Dataset_Cohérent_100"
os.makedirs(save_dir, exist_ok=True)

saine_dir = os.path.join(save_dir, "Variantes_Saines")
defaut_dir = os.path.join(save_dir, "Variantes_Defauts")

os.makedirs(saine_dir, exist_ok=True)
os.makedirs(defaut_dir, exist_ok=True)

# Sauvegarde de la machine de référence
ref_path = os.path.join(save_dir, "Tesla_Model3_Reference.json")
M_ref.save(ref_path)

saine_count = 0
defaut_count = 0

for Mi, nm in zip(valid_machines, valid_names):
    if "Saine" in nm:
        sub_dir = saine_dir
        saine_count += 1
    else:
        sub_dir = defaut_dir
        defaut_count += 1
    
    out_path = os.path.join(sub_dir, nm + ".json")
    Mi.save(out_path)

print(f"\n DATASET COHÉRENT CRÉÉ AVEC SUCCÈS!")
print(f"    Dossier: {os.path.abspath(save_dir)}/")
print(f"    Total machines: {len(valid_names) + 1}")
print(f"    Variantes saines: {saine_count}")
print(f"    Variantes défauts: {defaut_count}")
print(f"    Machine de référence: 1")

print("\n" + "=" * 80)
print("GÉNÉRATION TERMINÉE")
print("=" * 80)

In [ ]:
--------SIMULATION DES MACHINES ----------------------

In [ ]:

# === 1. MACHINE DE RÉFÉRENCE ===
print("=" * 80)
print("TESLA MODEL 3 - VARIATIONS (EXCENTRICITÉS)")
print("=" * 80)

print("\n ANALYSE MACHINE DE RÉFÉRENCE")
print("=" * 55)

# Caractéristiques de référence
airgap_ref = M_ref.stator.Rint - M_ref.rotor.Rext
turns_ref = M_ref.stator.winding.Ntcoil
rotor_radius = M_ref.rotor.Rext

print(f" Entrefer de référence: {airgap_ref*1000:.2f} mm")
print(f" Tours de référence: {turns_ref}")
print(f" Rayon rotor: {rotor_radius*1000:.2f} mm")

# === 2. CRÉATION DES DOSSIERS ===
base_dir = "MACHINE_TESLA_MODEL_3"
excent_dir = os.path.join(base_dir, "machine_defaut_excentricite")
os.makedirs(excent_dir, exist_ok=True)

print(f"\n Dossier créé : {excent_dir}")

# Sauvegarde de la machine de référence
ref_path = os.path.join(excent_dir, "Tesla_Model3_Reference.json")
M_ref.save(ref_path)
print(f"\n Machine de référence sauvegardée : {ref_path}")

# === 3. GÉNÉRATION DES VARIANTES AVEC EXCENTRICITÉS RÉALISTES ===
print("\n   Groupe X: Excentricités RÉALISTES (5% à 20% de l'entrefer)")

eccentricity_percentages = np.linspace(0.05, 0.20, 8)  # 8 paliers
eccentricity_variations = eccentricity_percentages * airgap_ref

machines = []
names = []

print(f"   Excentricités à générer:")
for i, (pct, ecc) in enumerate(zip(eccentricity_percentages, eccentricity_variations)):
    print(f"     {i+1}. {pct*100:.0f}% de l'entrefer = {ecc*1000:.3f} mm")

for i, (pct, ecc) in enumerate(zip(eccentricity_percentages, eccentricity_variations)):
    M = deepcopy(M_ref)

    print(f"\n   Machine {i+1}: Excentricité {pct*100:.0f}% ({ecc*1000:.3f} mm)")

    # 1. Excentricité statique (décalage rotor)
    if hasattr(M.rotor, 'x0'):
        M.rotor.x0 = ecc
    if hasattr(M.rotor, 'y0'):
        M.rotor.y0 = 0.0

    # 2. Vérification entrefer
    original_airgap = M.stator.Rint - M.rotor.Rext
    min_airgap = original_airgap - ecc
    max_airgap = original_airgap + ecc

    print(f"     Entrefer original: {original_airgap*1000:.3f} mm")
    print(f"     Entrefer min: {min_airgap*1000:.3f} mm")
    print(f"     Entrefer max: {max_airgap*1000:.3f} mm")

    if min_airgap <= 0:
        print("     ATTENTION: Entrefer négatif, variante rejetée.")
        continue
    elif min_airgap < 0.1e-3:
        print("    ATTENTION: Entrefer très petit (<0.1mm).")

    # 3. Validation géométrique
    try:
        if hasattr(M.stator, 'slot'):
            M.stator.slot.build_geometry()
        print("    Géométrie valide")
    except Exception as e:
        print(f"   Géométrie invalide: {e}")
        continue

    # Ajout aux listes
    machines.append(M)
    name = f"Tesla_Model3_Defaut_Excentricite_{pct*100:.0f}Pourcent"
    names.append(name)

    # Sauvegarde directe dans le dossier excentricité
    out_path = os.path.join(excent_dir, name + ".json")
    M.save(out_path)
    print(f"    Sauvegardée sous : {out_path}")

# === 4. RÉSUMÉ FINAL ===
print("\n" + "="*80)
print("RÉSUMÉ DES EXCENTRICITÉS RÉALISTES GÉNÉRÉES")
print("="*80)
print(f" Entrefer de référence: {airgap_ref*1000:.2f} mm")

for i, (pct, ecc) in enumerate(zip(eccentricity_percentages, eccentricity_variations)):
    print(f"   {i+1:2d}. {pct*100:.0f}% -> {ecc*1000:.3f} mm")

print(f"\n Total machines générées et sauvegardées: {len(machines)}")
print("="*80)
